<a href="https://colab.research.google.com/github/chfava/INF8111/blob/master/TP1/TP1_INF8111_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INF8111 - Fouille de données


## TP1 Automne 2019 - Preprocessing de tweets pour de l'analyse de sentiments

##### Membres de l'équipe:

    - Charles-Olivier Favreau (1789479)
    - Jean-Guillaume Langlois (1741803)
    

## Présentation

Twitter est un réseau social permettant aux utilisateurs de publier des informations et communiquer entre eux par le biais de messages, appelés tweets, pouvant contenir jusqu'à 280 caractères. Largement utilisé aujourd'hui, ce réseau peut être un outil pour des entreprises qui souhaitent évaluer l'avis de leurs clients.

Dans ce TP, on se met à la place d'une compagnie aérienne, qui souhaiterait détecter les tweets qui la mentionnent et analyser si ce sont des mentions positives ou négatives, en comparant leur résultat avec les autres compagnies.

Le *preprocessing* est une tâche cruciale en fouille de données. Elle permet de transformer les données brutes en un format adapté à l'application de méthodes de machine learning.

# I/ Analyse de sentiments (13 Pts)

Usuellement dans la littérature, la tâche d'extraire le sentiment d'un texte est appelé *sentiment analysis*.
Ici pour se faire, nous allons utiliser un modèle *bag-of-words* (BoW).

## 1. Installation

Pour ce TP, vous aurez besoin des librairies `numpy`, `sklearn` et `scipy` (que vous avez sans doute déjà), ainsi que la librairie `nltk`, qui est une libraire utilisée pour faire du traitement du language (Natural Language Processing, NLP)

Installez les libraires en question et exécutez le code ci-dessous :

In [1]:
# If you want, you can use anaconda and install after nltk library
# !pip install --user numpy
# !pip install --user sklearn
# !pip install --user scipy
# !pip install --user nltk

import sys
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("universal_tagset")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

## 2. Jeu de données

On utilise un jeu de donnée provenant de *Crowdflower's Data for Everyone library*: https://www.figure-eight.com/data-for-everyone/

Pour citer la source originale de la base :

    A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service").

Les compagnies incluses dans cette base de données sont Virgin America, United Airline, Southwest Airlines, jetBlue, USAirways, et American Airlines.

Dans le fichier zip du TP, vous trouverez le fichier *airline_tweets_database.csv*, qui est la base de données de tweets que nous allons manipuler.

Chaque ligne de ce fichier contient un tweet, avec plusieurs informations : l'identifiant du tweet, l'utilisateur, le contenu, le nombre de retweet... Ainsi que le label.

3 labels différents sont possibles dans ce dataset : *négatif*, *neutre* et *positif*, que l'on va représenter respectivement par 0, 1 et 2.

Pour ce TP, on ne va conserver ici que le texte et le label. On divise ensuite la base de données en 3 ensembles (entrainement/validation/test). Vous utiliserez l'ensemble d'entraînement et de validation pour cette partie, et l'ensemble de test à la partie suivante.

Le code ci-dessous permet de charger ces ensembles:

In [2]:
import csv
from sklearn.model_selection import train_test_split

def load_dataset(path):
    
    x = []
    y = []

    with open(path, 'r', newline='', encoding="latin-1") as csvfile:
        
        reader = csv.reader(csvfile, delimiter=',')
        
        # Taking the header of the file + the index of useful columns:
        header = next(reader)
        ind_label = header.index('airline_sentiment')
        ind_text = header.index('text')
        
        for row in reader:
            x.append(row[ind_text])
            
            label = row[ind_label]
            
            if label == "negative":
                y.append(0)
            elif label == "neutral":
                y.append(1)
            elif label == "positive":
                y.append(2)

        assert len(x) == len(y)

        return x, y


# Path of the dataset
path = "airline_tweets_database.csv"

X, y = load_dataset(path)

train_valid_X, test_X, train_valid_Y, test_Y = train_test_split(X, y, test_size=0.15, random_state=12)

train_X, valid_X, train_Y, valid_Y = train_test_split(train_valid_X, train_valid_Y, test_size=0.18, random_state=12)

print("Length of training set : ", len(train_X))
print("Length of validation set : ", len(valid_X))
print("Length of test set : ", len(test_X))


Length of training set :  10204
Length of validation set :  2240
Length of test set :  2196


## 3. Preprocessing

Nous allons ici implémenter la *tokenization* et le *stemming*, qui sont 2 étapes courantes de preprocessing en NLP. Ensuite, afin d'avoir un modèle qui s'adapte mieux au format de Twitter, nous ajouterons une étape spécifique supplémentaire.

### 3.1. Tokenization

Cette étape permet de séparer un texte en séquence de *tokens* (= jetons, ici des mots, symboles ou ponctuation).

Par exemple la phrase *"It's the student's notebook."* peut être séparé en liste de tokens de cette manière: ["it", " 's", "the", "student", " 's", "notebook", "."].

**De plus, tous les tokenizers ont également le rôle de mettre le texte en minuscule.**


##### Question 1. Implémentez les 2 tokenizers différents suivants: (0.5 Pts)

- Le **SpaceTokenizer**, qui est un tokenizer naïf qui sépare simplement en fonction des espaces.
- Le **NLTKTokenizer**, qui utilise la méthode du package *nltk* (https://www.nltk.org/api/nltk.html).


In [0]:
from nltk.tokenize import TweetTokenizer

class SpaceTokenizer(object):
    """
    It tokenizes the tokens that are separated by whitespace (space, tab, newline). 
    We consider that any tokenization was applied in the text when we use this tokenizer.

    For example: "hello\tworld of\nNLP" is split in ['hello', 'world', 'of', 'NLP']
    """

    def tokenize(self, text):
        # TODO

        # Have to return a list of tokens
        tokens = text.split()
        return tokens


class NLTKTokenizer(object):
    """
    This tokenizer uses the default function of nltk package (https://www.nltk.org/api/nltk.html) to tokenize the text.
    """
    def __init__(self):
        self.tokenizer = TweetTokenizer()
        
    def tokenize(self, text):
        # TODO

        # Have to return a list of tokens
        
        tokens = self.tokenizer.tokenize(text)
       
        return tokens


#### Testez les deux tokenizers. Quelles différences pouvez-vous constater?

In [4]:
spaceTokenizer = SpaceTokenizer()
nltkTokenizer = NLTKTokenizer()

print("SpaceTokenizer : ")
print(spaceTokenizer.tokenize(train_X[0]))
print("\n" + "NLTKTokenizer : ")
print(nltkTokenizer.tokenize(train_X[0]))




SpaceTokenizer : 
['@USAirways', 'tells', 'me', 'to', 'talk', 'to', '@AmericanAir', 'about', 'my', 'delayed', 'flights.', 'AA', 'tells', 'me', 'to', 'talk', 'to', 'US.', '#ihatemergers']

NLTKTokenizer : 
['@USAirways', 'tells', 'me', 'to', 'talk', 'to', '@AmericanAir', 'about', 'my', 'delayed', 'flights', '.', 'AA', 'tells', 'me', 'to', 'talk', 'to', 'US', '.', '#ihatemergers']


Comme il est possible d'observer, les deux tokenizer ne donnent pas exactement la même sortie. Le tokenizer de NLTK isole les signes de ponctuation (.) comme un token en soi. À l'inverse, le SpaceTokenizer ne sépare pas les signes de ponctuation des mots. Ainsi, plusieurs tokens contiennent des signes de ponctuation à la fin. 

### 3.2. Troncature (ou Stemming)

Dans les phrases "I should have bought a new shoes today" et "I spent too much money buying games", les mots "buying" et "bought" représentent la même idée. Considérer ces deux mots comme différents ne ferait qu'augmenter pour rien la complexité et la dimension du problème, ce qui peut avoir un impact négatif sur la performance globale. Ainsi, on peut donc plutôt une forme unique (comme la racine du mot) pour représenter ces deux mots de la même manière. Ce procédé de conversion de mots en racines permettant de réduire la dimension est appelé usuellement *stemming*, que l'on peut traduire par troncature.


#### Question 2. Récupérez les troncatures des tokens en utilisant l'attribut *stemmer* de la classe *Stemmer* (0.5 Pts) 

In [0]:
from nltk.stem.snowball import SnowballStemmer


class Stemmer(object):

    def __init__(self):
        self.stemmer = SnowballStemmer("english", ignore_stopwords=True)

    def stem(self, token):
        """
        token: a string that contain a token
        """
        # TODO
        
        # Have to return the stemmed token
        
        return self.stemmer.stem(token)


### 3.3. Twitter preprocessing

Parfois, appliquer uniquement ces deux étapes ne suffit pas, due aux particularités des données que nous manipulons, qui peuvent demander des étapes de preprocessing spécifiques afin d'obtenir un modèle plus adapté.

Couramment en NLP, un dictionnaire est utilisé pour stocker un ensemble de mots, et tous les mots n'appartenant pas au dictionnaire sont considérés comme inconnus. Ainsi, avec ce choix d'implémentation, la dimension de l'espace caractéristique du modèle est directement liée au nombre de mots du dictionnaire. Ainsi, pour des raisons de complexité mais aussi car les modèles à trop grande dimension peuvent souffrir du fléau de la dimensionnalité, il est préférable de réduire la taille de notre vocabulaire.

#### Question 3. Donnez, en expliquant brièvement, au moins deux exemples d'étapes de préprocessing qui permettent de réduire la taille du dictionnaire ici, puis implémentez-les.  (2.0 points)

Ces étapes de préprocessing doivent être en rapport aux charactéristiques spécifiques des données de Twitter. La suppression des mots vides ne compte pas comme une des deux étapes.

In [0]:
import string

class TwitterPreprocessing(object):
  
    

    def preprocess(self, tweet):
        """
        tweet: original tweet
        """
        # TODO : Write your preprocessing steps here.

        # return the preprocessed twitter
        
        #minuscule
        tokens = [token.lower() for token in tweet]
        
        #Normalization
        #example : b4 -> before
        #otw -> on the way
        #gooood -> good
        
        #A TERMINER
        
        #remplacer dates par <DATE>
        
        new_tweet = tokens
        
        return new_tweet


### 3.4.  Pipeline

Une pipeline permet d'exécuter séquentiellement toutes les étapes de preprocessing, pour transformer les données brutes en une version utilisable pour notre modèle. La *PreprocessingPipeline* a été implémenter pour appliquer à la suite le tokenizer, les troncatures et le preprocessing spécifique à Twitter. 

**N'hésitez pas à changer l'ordre des étapes de preprocessing si vous le souhaitez.**

In [0]:
class PreprocessingPipeline:

    def __init__(self, tokenization, twitterPreprocessing, stemming):
        """
        tokenization: enable or disable tokenization.
        twitterPreprocessing: enable or disable twitter preprocessing.
        stemming: enable or disable stemming.
        """

        self.tokenizer = NLTKTokenizer() if tokenization else SpaceTokenizer()
        self.twitterPreprocesser = TwitterPreprocessing(
        ) if twitterPreprocessing else None
        self.stemmer = Stemmer() if stemming else None

    def preprocess(self, tweet):
        """
        Transform the raw data

        tokenization: boolean value.
        twitterPreprocessing: boolean value. Apply the
        stemming: boolean value.
        """

        tokens = self.tokenizer.tokenize(tweet)

        if self.stemmer:
            tokens = list(map(self.stemmer.stem, tokens))

        if self.twitterPreprocesser:
            tokens = self.twitterPreprocesser.preprocess(tokens)

        return tokens


Test de la pipeline :

In [8]:
pipeline = PreprocessingPipeline(tokenization=True, twitterPreprocessing=True, stemming=True)
print(list(map(pipeline.preprocess, train_X[:1])))

[['@usairway', 'tell', 'me', 'to', 'talk', 'to', '@americanair', 'about', 'my', 'delay', 'flight', '.', 'aa', 'tell', 'me', 'to', 'talk', 'to', 'us', '.', '#ihatemerg']]


## 4. N-grams

Un n-gram est une séquence continue de *n* tokens dans un texte. Par exemple, les séquences *"nous a"* et *"la porte"* sont deux exemples de 2-grams de la phrase *"Il nous a dit au revoir en franchissant la porte."*. 1-gram, 2-gram et 3-gram sont respectivement appelés unigram, bigram et trigram. 

Voici la liste de tous les unigrams, bigrams et trigrams possible pour la phrase *"Il nous a dit au revoir en franchissant la porte."* :
- Unigram: ['Il', 'nous', 'a', 'dit', 'au', 'revoir', 'en', 'franchissant', 'la', 'porte']
- Bigram: ['Il nous', 'nous a', 'a dit', 'dit au', 'au revoir', 'revoir en', 'en franchissant', 'franchissant la', 'la porte']
- Trigram: ['Il nous a', 'nous a dit', 'a dit au', 'dit au revoir', 'au revoir en', 'revoir en franchissant', 'en franchissant la', 'franchissant la porte']


##### Question 4. Implementez les fonctions `bigram` et `trigram`. (1 Pt)

Vous devez résoudre cette question sans utiliser de libraire exterieur comme scikit-learn par exemple.

In [0]:
def bigram(tokens):
    """
    tokens: a list of strings
    """
    # TODO
    
    # This function returns the list of bigrams
    bigrams = []
    for tokenIndex in range (0, len(tokens) - 1):
      bigram = tokens[tokenIndex] + " " + tokens[tokenIndex + 1]
      bigrams.append(bigram)
    return bigrams


def trigram(tokens):
    """
    tokens: a list of strings
    """
    # TODO
    
    trigrams = []
    for tokenIndex in range (0, len(tokens) - 2):
      trigram = tokens[tokenIndex]  + " " + tokens[tokenIndex + 1] + " " + tokens[tokenIndex + 2]
      trigrams.append(trigram)
    
    # This function returns the list of trigrams
    return trigrams


## 5. Bag-of-words

Régressions logistiques, SVM et d'autres modèles très courants demande des entrées qui soient toutes de la même taille, ce qui n'est forcément le cas pour des types de données comme les textes, qui peuvent avoir un nombre variable de mots.  

Par exemple, considérons la phrase 1, ”Board games are much better than video games” et la phrase 2, ”Pandemic is an awesome game!”. La table ci-dessous montre un exemple d'un moyen de représentation de ces deux phrases en utilisant une représentation fixe : 

|<i></i>     | an | are | ! | pandemic | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 0 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |


Chaque colonne représente un mot du vocabulaire (de longueur 13), tandis que chaque ligne contient l'occurence des mots dans une phrase. Ainsi, la valeur 2 à la position (1,7) est due au fait que le mot *"games"* apparait deux fois dans la phrase 1. 

Ainsi, chaque ligne étant de longueur 13, on peut les utiliser comme vecteur pour représenter les phrases 1 et 2. Ainsi, c'est cette méthode que l'on appelle *Bag-of-Words* : c'est une représentation de documents par des vecteurs dont la dimension est égale à la taille du vocabulaire, et qui est construit en comptant le nombre d'occurence de chaque mot. Ainsi, chaque token est ici associé à une dimension.


##### Question 5. Implémentez le Bag-of-Words (2 Pts)

Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe sparse.csr_matrix de scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).

In [0]:
from scipy.sparse import csr_matrix
import operator

class CountBoW(object):

    def __init__(self, pipeline, bigram=False, trigram=False):
        """
        pipelineObj: instance of PreprocesingPipeline
        bigram: enable or disable bigram
        trigram: enable or disable trigram
        words: list of words in the vocabulary
        """
        self.pipeline = pipeline
        self.bigram = bigram
        self.trigram = trigram
        self.words = None

    def computeBoW(self, tokens):
        """
        Calcule du BoW, à partir d'un dictionnaire de mots et d'une liste de tweets.
        On suppose que l'on a déjà collecté le dictionnaire sur l'ensemble d'entraînement.
        
        Entrée: tokens, une liste de vecteurs contenant les tweets (une liste de liste)
        
        Return: une csr_matrix
        """
        
        if self.words is None:
            raise Exception(
                "fit_transform() should be called first (no dictionnary available)"
            )
        
        # TODO
        
        bagOfWords = csr_matrix((len(tokens), len(self.words)), dtype=np.int8).toarray()
        
        for tweet in range (0, len(tokens)):
          for word in range(0, len(tokens[tweet])):
            if(self.bigram):
              if(word + 1 < len(tokens[tweet])):
                bigramCombo = tokens[tweet][word] + " " + tokens[tweet][word + 1]
                if(bigramCombo in self.words):
                  bagOfWords[tweet, self.words.index(bigramCombo)] += 1
            if(self.trigram):
              if(word + 2 < len(tokens[tweet])):
                trigramCombo = tokens[tweet][word] + " " + tokens[tweet][word + 1] + " " + tokens[tweet][word + 2]
                if(trigramCombo in self.words):
                  bagOfWords[tweet, self.words.index(trigramCombo)] += 1
            if(tokens[tweet][word] not in self.words):
              bagOfWords[tweet, self.words.index("<UNK>")] += 1
            else:
              bagOfWords[tweet, self.words.index(tokens[tweet][word])] += 1
              
        
        return bagOfWords

    def fit_transform(self, X):
        """
        Cette méthode preprocess les données en utilisant la pipeline, ajoute les bigram et trigram 
        si besoin, et transforme les textes en vecteurs d'entiers.
        
        Entrée: X, une liste de vecteurs contenant les tweets
        
        Return: une csr_matrix
        """
        # TODO
        X_transformed = []
        for tweet in X:
          X_transformed.append(pipeline.preprocess(tweet))
          
        if(self.bigram):
          bigrams = []
          for tweet in X_transformed:
            bigrams.append(bigram(tweet))
        
        if(self.trigram):
          trigrams = []
          for tweet in X_transformed:
            trigrams.append(trigram(tweet))
        
        
        
        #dictionnaire
        dictionary = {}
        for tweet in range (0, len(X_transformed)):
          for word in range(0, len(X_transformed[tweet])):
            if(X_transformed[tweet][word] not in dictionary):
              dictionary[X_transformed[tweet][word]] = 1
            else:
              dictionary[X_transformed[tweet][word]] = dictionary[X_transformed[tweet][word]] + 1
              
              
        if(self.bigram):
          for tweet in range (0, len(bigrams)):
            for word in range(0, len(bigrams[tweet])):
              if(bigrams[tweet][word] not in dictionary):
                dictionary[bigrams[tweet][word]] = 1
              else:
                dictionary[bigrams[tweet][word]] = dictionary[bigrams[tweet][word]] + 1
                
        if(self.trigram):
          for tweet in range (0, len(trigrams)):
            for word in range(0, len(trigrams[tweet])):
              if(trigrams[tweet][word] not in dictionary):
                dictionary[trigrams[tweet][word]] = 1
              else:
                dictionary[trigrams[tweet][word]] = dictionary[trigrams[tweet][word]] + 1
        
        sortedArray = sorted(dictionary.items(), key=operator.itemgetter(1), reverse=True)[:5000]
        sorted_keys = [ item[0] for item in sortedArray ]
        self.words = sorted_keys
        
        #enlever les mots qui ne sont pas dans le dictionnaire
        '''
        for tweet in range (0, len(X_transformed)):
          for word in range(0, len(X_transformed[tweet])):
            if(X_transformed[tweet][word] not in self.words):
              X_transformed[tweet][word] = "<UNK>"
        '''
        self.words.append("<UNK>")
        
        bOfW = self.computeBoW(X_transformed)
        
        return bOfW
        
        
    
    def transform(self, X):
        """
        Cette méthode preprocess les données en utilisant la pipeline, ajoute les bigram et trigram 
        si besoin, et transforme les textes en vecteurs d'entiers.
        Différence avec fit_transform : on suppose qu'on dispose déjà du dictionnaire ici

        Entrée: X, une liste de vecteurs contenant les tweets
        
        Return: une csr_matrix
        """
        
        if self.words is None:
            raise Exception(
                "fit_transform() should be called first (no dictionnary available)"
            )

        # TODO
        
        X_transformed = []
        for tweet in X:
          X_transformed.append(pipeline.preprocess(tweet))
        
        bOfW = self.computeBoW(X_transformed)
        
        return bOfW


## 6. TF-IDF

L'utilisation de la fréquence d'apparition brute des mots, comme c'est le cas avec le bag-of-words, peut être problématique. En effet, peu de tokens auront une fréquence très élevée dans un document, et de ce fait, le poids de ces mots sera beaucoup plus grand que les autres, ce qui aura tendance à biaiser l'ensemble des poids. De plus, les mots qui apparaissent dans la plupart des documents n'aident pas à les discriminer. Par exemple, le mot "*de*" apparaît dans beaucoup de tweets de la base de données, et pour autant, avoir ce mot en commun ne permet pas de conclure que des tweets sont similaires. Au contraire, le mot "*génial*" est plus rare, mais les documents qui contiennent ce mot sont plus susceptibles d'être positif. TF-IDF est donc une méthode qui permet de pallier à ce problème.

TF-IDF pondère le vecteur en utilisant une fréquence de document inverse (IDF) et une fréquence de termes (TF).

TF est l'information locale sur l'importance qu'a un mot dans un document donné, tandis que IDF mesure la capacité de discrimination des mots dans un jeu de données. 

L'IDF d'un mot se calcule de la façon suivante:

\begin{equation}
	\text{idf}_i = \log\left( \frac{N}{\text{df}_i} \right),
\end{equation}

avec $N$ le nombre de documents dans la base de donnée, et $\text{df}_i$ le nombre de documents qui contiennent le mot $i$.

Le nouveau poids $w_{ij}$ d'un mot $i$ dans un document $j$ peut ensuite être calculé de la façon suivante:

\begin{equation}
	w_{ij} = \text{tf}_{ij} \times \text{idf}_i,
\end{equation}

avec $\text{tf}_{ij}$ la fréquence du mot $i$ dans le document $j$.




##### Question 6. Implémentez le bag-of-words avec la pondération de TF-IDF (3 Pts)

Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe sparse.csr_matrix de scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).

In [0]:
from scipy.sparse import csr_matrix
import math


class TFIDFBoW(object):

    def __init__(self, pipeline, bigram=False, trigram=False):
        """
        pipelineObj: instance of PreprocesingPipeline
        bigram: enable or disable bigram
        trigram: enable or disable trigram
        words: list of words in the vocabulary
        idf: list of idfs for each document
        """
        self.pipeline = pipeline
        self.bigram = bigram
        self.trigram = trigram
        self.words = None
        self.idf = None
    
    def computeTFIDF(self, X):
        """
        Calcule du TF-IDF, à partir d'un dictionnaire de mots et d'une 
        liste de tweets.
        On suppose que l'on a déjà collecté le dictionnaire ainsi que 
        calculé le vecteur contenant l'idf pour chaque document.
        
        Entrée : X, une liste de vecteurs contenant les tweets
        
        Return: une csr_matrix
        """
        
        if self.words is None:
            raise Exception(
                "fit_transform() should be called first (no dictionnary available)"
            )
        
        # TODO


    def fit_transform(self, X):
        """
        Cette méthode preprocess les données en utilisant la pipeline, ajoute les bigram et trigram 
        si besoin, et transforme les textes en vecteurs de flottants avec la pondération TF-IDF.
        
        Entrée : X, une liste de vecteurs contenant les tweets
        
        Return: une csr_matrix
        """
        # TODO

    def transform(self, X):
        """
        Cette méthode preprocess les données en utilisant la pipeline, ajoute les bigram et trigram 
        si besoin, et transforme les textes en vecteurs de flottants avec la pondération TF-IDF.
        Différence avec fit_transform : on suppose qu'on dispose déjà du dictionnaire et du calcul des idf ici.
            
        Entrée : X, une liste de vecteurs contenant les tweets
        
        Return: une csr_matrix
        """

        if self.words is None:
            raise Exception(
                "fit_transform() should be called first (no dictionnary available)"
            )

        # TODO
    

## 7. Classification utilisant BoW

Pour la classification, nous allons effectuer une régression logisitique (vu en cours ou que vous allez voir bientôt). 
Pour en savoir plus : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

La méthode `train_evaluate` entraîne et évalue le modèle de régression logistique.


In [0]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


def train_evaluate(training_X, training_Y, validation_X, validation_Y, bowObj):
    """
    training_X: tweets from the training dataset
    training_Y: tweet labels from the training dataset
    validation_X: tweets from the validation dataset
    validation_Y: tweet labels from the validation dataset
    bowObj: Bag-of-word object
    
    :return: the classifier and its accuracy in the training and validation dataset.
    """

    classifier = LogisticRegression(n_jobs=-1)

    training_rep = bowObj.fit_transform(training_X)

    print(training_rep)
    classifier.fit(training_rep, training_Y)

    trainAcc = accuracy_score(training_Y, classifier.predict(training_rep))
    validationAcc = accuracy_score(
        validation_Y, classifier.predict(bowObj.transform(validation_X)))

    return classifier, trainAcc, validationAcc



##### Question 7. Entraînez et calculez la précision de la régression logistique sur les ensembles d'entraînement et de validation. (4 points)

Essayez les configurations suivantes :

    1. CountBoW + SpaceTokenizer(without tokenizer) + unigram 
    2. CountBoW + NLTKTokenizer + unigram
    3. TFIDFBoW + NLTKTokenizer + Stemming + unigram
    4. TFIDFBoW + NLTKTokenizer + Twitter preprocessing + Stemming  + unigram
    5. TFIDFBoW + NLTKTokenizer + Twitter preprocessing + Stemming  + unigram + bigram
    6. TFIDFBoW + NLTKTokenizer + Twitter preprocessing + Stemming  + unigram + bigram + trigram

Outre la précision, reportez la taille du dictionnaire pour chacune des configurations. Enfin, décrivez vos résultats obtenus et répondez aux questions suivantes:
- Quelles étapes de preprocessing ont effectivement aidé le modèle ? Pourquoi ?
- La pondération avec TF-IDF a-t-elle aidé à obtenir une meilleure performance que le simple BoW ?
- Les bigrams et trigrams ont-ils amélioré la performance ? Expliquez pourquoi.

Indiquez quelle est la configuration que vous choisissez.

In [38]:
bagOfWords = CountBoW(pipeline, bigram=True, trigram=True)
train_evaluate(train_X, train_Y, valid_X, valid_Y, bagOfWords)



[[2 4 0 ... 0 0 1]
 [0 0 0 ... 0 0 2]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 3]
 [1 0 0 ... 0 0 4]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='warn', n_jobs=-1, penalty='l2',
                    random_state=None, solver='warn', tol=0.0001, verbose=0,
                    warm_start=False), 0.9494315954527637, 0.8044642857142857)

# II/ Prototype (7 points)

Maintenant que nous avons un modèle de classification entraîné pour l'analyse de sentiments, nous pouvons l'appliquer à notre ensemble de tests et analyser le résultat.

## 1. Analyse de Sentiments


##### Question 9. Implémentez la fonction `detect_airline` qui détecte la compagnie aérienne d'un tweet. (1,5 points)

Expliquez votre approche, et les inconvénients possibles.

**Attention :** `detect_airline` doit être en mesure de gérer le cas où aucune compagnie n'est mentionnée (auquel cas `None` est retounée), mais aussi le cas où plusieurs compagnies sont mentionnées dans un tweet.

In [0]:
def detect_airline(tweet):
    """
    Detect and return the airline companies mentioned in the tweet
    
    tweet: represents the tweet message. You should define the data type
    
    Return: list of detected airline companies
    """
    
    # TODO



##### Question 10. Implémentez la fonction `extract_sentiment` qui, à partir de tweets et d'un classificateur, extrait leurs sentiments. (0.5 points)


In [0]:
def extract_sentiment(classifier, tweets):
    """
    Extract the tweet sentiment
    
    classifier: classifier object
    tweet: represents the tweet message. You should define the data type
    
    Return: list of detected airline companies
    """
    # TODO


##### Question 11. En utilisant `extract_tweet_content`, `detect_airline` et `extract_sentiment`, générez un diagramme en bar contenant le nombre de tweets positives, négatifs et neutres pour chacune des compagnies. (2 points)

Décrivez brièvement le diagramme et analysez les résultats (par exemple, quelle est la compagnie avec le plus de tweets négatifs?). Expliquez comment un tel diagramme peut aider des compagnies aériennes.

## 2. Analyse de termes

Le POS-tagging (pour *part-of-speech tagging*, en français étiquetage grammatical) consiste à l'extraction de l'information grammaticale d'un token dans une phrase. Par exemple, la table ci-dessous donne un exemple du *POS-tagging* de la phrase *"The cat is white!"*


|   The   | cat  |  is  | white     |    !       |
|---------|------|------|-----------|------------|
| article | noun | verb | adjective | punctation |


Pour autant, le *POS-tagging* peut être plus complexe que les règles simples apprises à l'école. Il faut souvent des informations plus détaillées sur le rôle d'un terme dans une phrase. Pour notre problème, nous n'avons pas besoin d'utiliser un modèle linguistique plus complexe, nous allons utiliser ce qu'on appelle des *POS-tags* universelles.

En *POS-tagging*, chaque token est représenté par un tag. La liste des POS-tags utilisés sont disponibles ici :
https://universaldependencies.org/u/pos/ .

In [0]:
# NLTK POS-tagger

import nltk


#before using pos_tag function, you have to tokenize the sentence.
s = ['The', 'cat', 'is',  'white', '!']
nltk.pos_tag(s,tagset='universal')


##### Question 12. Implémentez un code qui collecte les 10 termes les plus fréquents pour chaque compagnie aérienne. (2 Pts)

Ici, vous n'allez considérer que les termes apparaissant dans les tweets positifs et négatifs. 

De plus, nous allons utiliser la définition suivante de "terme":

1. Un mot qui est soit un adjectif, soit un nom
2. Un N-gram composé d'adjectifs suivit par un nom (par exemple, "nice place"), ou un nom suivi par un autre nom (par exemple, "sports club").

Ensuite, **générez une table** contenant les 10 termes les plus fréquents, avec leurs fréquences (en pourcentage) pour chaque compagnie.

*N'oubliez pas de supprimer le nom de la compagnie parmi les termes !*


##### Question 13. Que conclure de la table généré à la question 12 pour les compagnies ? (1 Pt)


# III/ Bonus (2 points)

Les noms de personnes, les noms de sociétés et les emplacements sont appelés "entités nommées". La reconnaissance d'entité nommée (NER, pour *Named-entity recognition*) consiste à extraire les entités nommées en les classant à l'aide de catégories prédéfinies. Dans cette section bonus, vous utiliserez un outil de NER pour extraire automatiquement des entités nommées des tweets. Cette approche est suffisamment générique pour récupérer des informations sur d’autres sociétés ou même des noms de produits et de personnes.


**Pour le bonus, vous êtes libres d'utiliser n'importe quel NER implémenté en Python.**

##### Question Bonus 1.  Implémentez un code qui génère une table contenant le top 10 des NER de la base de données. (1 point)

Cette table doit contenir les fréquences des entités nommées. Ensuite, générez un diagramme en bar qui montre le nombre de tweets positifs, négatifs ou neutres pour chacunes des 10 NER. Décrivez le résultat obtenu.


##### Question Bonus 2. Générez une table similaire à la question 12 pour le top 10 des NER pour chaque compagnie. (1 point)

Que peut-on conclure de ces résultats?